### Eksperymenty

**Modele**
- resnet50
- vgg16

**Zbiory danych**
- CIFAR100
- SVHN

**Metody**
- Random
- KMeans
- KMeansPurity
- KMeansDino
- KMeansDinoPurity

**Ustawienia**
- ratio = (0.1, 1, 10)
- epochs = 10
- batch_size = 512
- clip = 5.0
- num_clusters = 10
- eqsize = True
- min_purity = 0.1 (SVHN), 0.01 (CIFAR100)

**Nazewnictwo przebiegów**
W nawiasach kwadratowych podano wartość do wyboru ("jeden z ...") w nawiasch klamrowych wartości opcjonalne.
ID służy do rozróżniania eksperymentów w tej samej konfiguracji

- [r50,vgg13],ep:10,bs:512,clip:5.0,{nc:20},{eqsize},{mp:[0.1,0.01]},ID:[1,2,3,..]
- przykład: r50,ep:10,bs:512,clip:5.0,ID:1
- przykład: r50,ep:10,bs:512,clip:50,nc:20,eqsize,mp:0.1,ID:1

**Uwagi**
- Przy każdym eksperymencie nowy model (!)

In [ ]:
import torch
import torchvision
from torchvision.models import resnet50

from kiss.models import vgg16_kiss
from kiss.experiment import Experiment
from kiss.sampler import RandomSampler, KMeansSampler, KMeansPuritySampler, KMeansDinoSampler, KMeansPurityDinoSampler
from kiss.utils.configs import CONFIGS

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
])

dataset_cifar10_tr = torchvision.datasets.CIFAR10(root='../data/cifar10', train=True, download=True, transform=transform)
dataset_cifar10_te = torchvision.datasets.CIFAR10(root='../data/cifar10', train=False, download=True, transform=transform)

dataset_cifar100_tr = torchvision.datasets.CIFAR100(root='../data/cifar100', train=True, download=True, transform=transform)
dataset_cifar100_te = torchvision.datasets.CIFAR100(root='../data/cifar100', train=False, download=True, transform=transform)

dataset_svhn_tr = torchvision.datasets.SVHN(root='../data/svhn', split='train', download=True, transform=transform)
dataset_svhn_te = torchvision.datasets.SVHN(root='../data/svhn', split='test', download=True, transform=transform)

In [ ]:
ID = 6
RATIO = (0.1, 1, 10)
EPOCHS = 10
BATCH_SIZE = 512
CLIP = 5.0
NUM_CLUSTERS = 10
EQSIZE = True

In [3]:
import os

for model_fun in [resnet50, vgg16_kiss]:
    for dataset_tr, dataset_te in zip([dataset_cifar100_tr, dataset_svhn_tr], [dataset_cifar100_te, dataset_svhn_te]):
# for model_fun in [vgg16_kiss]:
    # for dataset_tr, dataset_te in zip([dataset_cifar100_tr], [dataset_cifar100_te]):
        for sampler_cls in [RandomSampler, KMeansSampler, KMeansDinoSampler, KMeansPuritySampler, KMeansPurityDinoSampler]:

            if model_fun.__name__ == 'resnet50':
                MODEL_NAME = 'r50'
            if model_fun.__name__ == 'vgg16':
                MODEL_NAME = 'vgg16'
            if model_fun.__name__ == 'vgg16_kiss':
                MODEL_NAME = 'vgg16_kiss'

            RUN_NAME = f"{MODEL_NAME},ep:{EPOCHS},bs:{BATCH_SIZE},clip:{CLIP}"

            if dataset_tr.__class__.__name__ == 'SVHN':
                NUM_CLASSES = 10
                MIN_PURITY = 0.1
            if dataset_tr.__class__.__name__ == 'CIFAR100':
                NUM_CLASSES = 100
                MIN_PURITY = 0.01

            if 'KMeans' in sampler_cls.__name__:
                RUN_NAME += f",nc:{NUM_CLUSTERS}"

            if 'KMeans' in sampler_cls.__name__ and EQSIZE:
                RUN_NAME += ",eqsize"

            if 'KMeansPurity' in sampler_cls.__name__:
                RUN_NAME += f",mp:{MIN_PURITY}"

            SAVE_CLUSTERS = LOAD_CLUSTERS = f"../checkpoints/{sampler_cls.__name__},{dataset_tr.__class__.__name__},nc:{NUM_CLUSTERS}"
            
            if not os.path.exists(LOAD_CLUSTERS):
                LOAD_CLUSTERS = None

            RUN_NAME += f',ID:{ID}'

            model = model_fun(num_classes=NUM_CLASSES)
            model.to(torch.device(CONFIGS.torch.device))

            experiment = Experiment(
                model = model,
                dataset_tr = dataset_tr,
                dataset_te = dataset_te,
                sampler_cls=sampler_cls,
                ratio=RATIO,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                clip=CLIP,
                num_clusters=NUM_CLUSTERS,
                eqsize=EQSIZE,
                min_purity=MIN_PURITY,
                load_clusters=LOAD_CLUSTERS,
                save_clusters=SAVE_CLUSTERS,
            )
            experiment.run("../experiments", RUN_NAME)

Validating: 100%|██████████| 29/29 [00:02<00:00,  9.72 batch/s, loss=0.3728]


Best valid loss improved. Current accuracy is 89.68%. Saving checkpoint...
Best valid accuracy improved. Current accuracy is 89.68%. Saving checkpoint...


Validating: 100%|██████████| 29/29 [00:02<00:00, 10.02 batch/s, loss=0.3649]


Best valid loss improved. Current accuracy is 90.23%. Saving checkpoint...
Best valid accuracy improved. Current accuracy is 90.23%. Saving checkpoint...


Validating: 100%|██████████| 29/29 [00:02<00:00,  9.92 batch/s, loss=0.3968]


...

Validating: 100%|██████████| 29/29 [00:02<00:00, 10.09 batch/s, loss=0.2592]


Best valid accuracy improved. Current accuracy is 93.84%. Saving checkpoint...


Validating: 100%|██████████| 29/29 [00:02<00:00, 10.09 batch/s, loss=0.2990]


Best valid accuracy improved. Current accuracy is 93.93%. Saving checkpoint...


Testing: 100%|██████████| 51/51 [00:05<00:00,  9.82 batch/s]
